# Screencast

In this Notebook I´ll be working on the question if there income of female and male professional developers is gender-biased.
I´ll do this by performing a permutation test
The stackoverflow Data from 2020 is used.

First I'll read in the necessary libraries for data wrangling and building plots. Moreover I´ll read in the csv file and hava a first look at the data in the dataset.

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

df = pd.read_csv('/Users/anna/Downloads/developer_survey_2020/survey_results_public.csv')
df.head()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,...,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,...,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,...,NaN,NaN,No,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat less welcome now than last year,40.0,7,4
4,5,"I used to be a developer by profession, but no...",Yes,31.0,16,NaN,NaN,NaN,United States,NaN,...,Easy,Too short,No,"Computer science, computer engineering, or sof...",Django;Ruby on Rails,Ruby on Rails,Just as welcome now as I felt last year,NaN,15,8


In [2]:
# Filtering the dataset, to only get professional developers that 
# belong to the Gender 'male' or 'female'
df=pd.DataFrame(df.query("MainBranch == 'I am a developer by profession' and (Gender=='Man' or Gender=='Woman')"))

# dropping NaNs of convertedComp column
df=df.dropna(subset=['ConvertedComp'])

In [3]:
# Splitting the data into male,female and complete datasets and filtering only for ConvertedComp  

data_male= df[df['Gender']=='Man'].ConvertedComp
data_female= df[df['Gender']=='Woman'].ConvertedComp
data_concat=df.ConvertedComp

My Nullhypothesis is that gender has no influence on the income of professional developers.

I will perform a two sample hypothesis test for difference of medians.
This means I´ll check how likely it is that male and female developers have the same mean salary without having the same distribution.
To do the two-sample bootstrap test, I shift both arrays to have the same mean, since we are simulating the hypothesis that their means are, in fact, equal. We then draw bootstrap samples out of the shifted arrays and compute the difference in means. This constitutes a bootstrap replicate, and we generate many of them. The p-value is the fraction of replicates with a difference in means greater than or equal to what was observed.

First I´ll define a function to perform the bootstrap replication.
Next I'll perform the analysis 

In [10]:
def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = func(np.random.choice(data,len(data)))

    return bs_replicates



In [14]:
# Compute difference of mean impact force from experiment: empirical_diff_means
empirical_diff_means = np.mean(data_male)-np.mean(data_female)

# Compute mean of all forces: mean_force
mean_force = np.mean(data_concat)

# Generate shifted arrays
data_male_shifted = data_male - np.mean(data_male) + mean_force
data_female_shifted = data_female-np.mean(data_female)+mean_force 

# Compute 10,000 bootstrap replicates from shifted arrays
bs_replicates_male = draw_bs_reps(data_male_shifted, np.mean, size=10000)
bs_replicates_female = draw_bs_reps(data_female_shifted, np.mean, size=10000)

# Get replicates of difference of means: bs_replicates
bs_replicates = bs_replicates_male-bs_replicates_female

# Compute and print p-value: p
p = np.sum(bs_replicates >= empirical_diff_means) / len(bs_replicates)
print('p-value =', p)

p-value = 0.085


A p-value higher than 0.05 (> 0.05) is not statistically significant and indicates strong evidence for the null hypothesis.
Here this means, there is no gender-bias in compensation.
Nevertheless due to the relatively small p-value i would recommend further analysis